In [1]:
%matplotlib notebook
#%matplotlib inline

import json
from collections import OrderedDict
import copy
import numpy as np
import matplotlib.pyplot as plt
from _converter import SensorThings2Dict
from _plotter import plot_confusion_matrix

In [5]:
with open("ABU1.txt") as f:
    data = []
    bad = 0
    for line in f:
        try:
            features = SensorThings2Dict(line)
            data.append(list(features.values()))
        except Exception, e:
            bad+=1
print("Incomplete rows: {}".format(bad))
print("Loaded: {}".format(len(data)))

Incomplete rows: 65
Loaded: 9484


In [7]:
data = np.asarray(data)
#print(data[:,2:-1])
# numerics = data[:,2:-1]
# for n in numerics[data[:,-1]=='True']:
#     plt.plot(n);
mask = np.random.rand(len(data)) < 0.9
train = data[mask]
test = data[~mask]
print("Train Total: {} Good: {} Faulty: {} Ratio: {}".format(len(train), len(train[train[:,-1]=='True']), len(train[train[:,-1]=='False']), float(len(train[train[:,-1]=='False']))/len(train)))
print("Test  Total: {} Good: {} Faulty: {} Ratio: {}".format(len(test), len(test[test[:,-1]=='True']), len(test[test[:,-1]=='False']), float(len(test[test[:,-1]=='False']))/len(train)))

Train Total: 8521 Good: 8041 Faulty: 480 Ratio: 0.0563314165004
Test  Total: 963 Good: 901 Faulty: 62 Ratio: 0.00727614129797


In [19]:
""" Balance data """
def balanceData():
    global train
    faulty = train[train[:,-1]=='False']
    not_faulty = train[train[:,-1]=='True']
    train = np.concatenate((not_faulty, np.repeat(faulty, 15, axis=0))) # repeate faulties
    #train = np.concatenate((not_faulty[:len(faulty)*2], faulty))
    print("Train Total: {} Good: {} Faulty: {} Ratio: {}".format(len(train), len(train[train[:,-1]=='True']), len(train[train[:,-1]=='False']), float(len(train[train[:,-1]=='False']))/len(train)))
    train = train[np.random.permutation(train.shape[0])] # shuffle data
balanceData()
#train_data = train[train[:,-1]=='True']

Train Total: 15241 Good: 8041 Faulty: 7200 Ratio: 0.472409946854


In [20]:
train_data = train[:,2:-1].astype(np.float32)
test_data = test[:,2:-1].astype(np.float32)
"""
Quality_OK is mapped to Faultiness
    'False' -> 1 (Faulty)
    'True'  -> 0 (Good)
"""
train_labels = np.array(train[:,-1]=='False').astype(np.int32)
test_labels = np.array(test[:,-1]=='False').astype(np.int32)

In [9]:
def eval_metrics(expected, predicted):
    print("------------------------- EVALUATION -------------------------")
    print("Accuracy Score: {}".format(metrics.accuracy_score(expected, predicted)))
    print(metrics.classification_report(expected, predicted))
    print("Confusion Matrix:")
    print(metrics.confusion_matrix(expected, predicted))
    print("Kappa Score: {}".format(metrics.cohen_kappa_score(expected, predicted)))
#     print("Matthews Correlation Coefficient: {}".format(metrics.matthews_corrcoef(expected, predicted)))
    print("--------------------------------------------------------------")
#     plot_confusion_matrix(metrics.confusion_matrix(expected, predicted),classes=['Good','Faulty'],title='Confusion matrix',normalize=True)
#     plt.show()

In [10]:
""" Classification and Regression Trees (CART) """
from sklearn.tree import DecisionTreeClassifier
from sklearn import metrics

clf = DecisionTreeClassifier(criterion="gini", random_state=1, max_depth=100,  class_weight={0:1.0, 1:0.05})
clf.fit(train_data, train_labels)

eval_metrics(train_labels, clf.predict(train_data))
eval_metrics(test_labels, clf.predict(test_data))
# from sklearn.tree import export_graphviz
# export_graphviz(clf, out_file='tree.dot') 

------------------------- EVALUATION -------------------------
Accuracy Score: 0.999647928647
             precision    recall  f1-score   support

          0       1.00      1.00      1.00      8041
          1       1.00      0.99      1.00       480

avg / total       1.00      1.00      1.00      8521

Confusion Matrix:
[[8041    0]
 [   3  477]]
Kappa Score: 0.996678696534
--------------------------------------------------------------
------------------------- EVALUATION -------------------------
Accuracy Score: 0.998961578401
             precision    recall  f1-score   support

          0       1.00      1.00      1.00       901
          1       1.00      0.98      0.99        62

avg / total       1.00      1.00      1.00       963

Confusion Matrix:
[[901   0]
 [  1  61]]
Kappa Score: 0.99131532669
--------------------------------------------------------------


In [13]:
""" Random Forest """
from sklearn.ensemble import RandomForestClassifier
from sklearn import metrics

clf = RandomForestClassifier(n_estimators=100,  class_weight={0:1.0, 1:0.05}, warm_start=True)

clf = clf.fit(train_data, train_labels)
eval_metrics(train_labels, clf.predict(train_data))
eval_metrics(test_labels, clf.predict(test_data))

------------------------- EVALUATION -------------------------
Accuracy Score: 1.0
             precision    recall  f1-score   support

          0       1.00      1.00      1.00      8041
          1       1.00      1.00      1.00       480

avg / total       1.00      1.00      1.00      8521

Confusion Matrix:
[[8041    0]
 [   0  480]]
Kappa Score: 1.0
--------------------------------------------------------------
------------------------- EVALUATION -------------------------
Accuracy Score: 0.996884735202
             precision    recall  f1-score   support

          0       1.00      1.00      1.00       901
          1       1.00      0.95      0.98        62

avg / total       1.00      1.00      1.00       963

Confusion Matrix:
[[901   0]
 [  3  59]]
Kappa Score: 0.973545651836
--------------------------------------------------------------


In [ ]:
import random
from copy import deepcopy
def randomPair(predicted_label):
    index = random.sample(np.where(train_labels==abs(1-predicted_label))[0],1)
    return deepcopy(train_data[index][0]), deepcopy(train_labels[index][0])

In [ ]:
for i,t in enumerate(test_data):
    prediction = clf.predict([t])[0]
    pair_data, pair_label = randomPair(prediction)
#     print(test_data[i])
#     print(pair_data)
    clf = clf.fit([test_data[i],pair_data], [prediction,pair_label])

In [ ]:
eval_metrics(train_labels, clf.predict(train_data))
eval_metrics(test_labels, clf.predict(test_data))

In [12]:
""" Bagging Classifier """ 
from sklearn.ensemble import BaggingClassifier
from sklearn import metrics

clf = BaggingClassifier(n_estimators=50, random_state=1)
clf = clf.fit(train_data, train_labels)

eval_metrics(train_labels, clf.predict(train_data))
eval_metrics(test_labels, clf.predict(test_data))

------------------------- EVALUATION -------------------------
Accuracy Score: 1.0
             precision    recall  f1-score   support

          0       1.00      1.00      1.00      8041
          1       1.00      1.00      1.00       480

avg / total       1.00      1.00      1.00      8521

Confusion Matrix:
[[8041    0]
 [   0  480]]
Kappa Score: 1.0
--------------------------------------------------------------
------------------------- EVALUATION -------------------------
Accuracy Score: 0.994807892004
             precision    recall  f1-score   support

          0       1.00      1.00      1.00       901
          1       0.95      0.97      0.96        62

avg / total       0.99      0.99      0.99       963

Confusion Matrix:
[[898   3]
 [  2  60]]
Kappa Score: 0.957223954585
--------------------------------------------------------------


In [21]:
""" MLP """
from sklearn.neural_network import MLPClassifier
from sklearn import metrics

solvers = ['lbfgs', 'sgd', 'adam']
clf = MLPClassifier(solver=solvers[0], alpha=1e-5, hidden_layer_sizes=(25,50,25), random_state=1, 
                    max_iter=1000, verbose=True, tol=1e-6)
clf = clf.fit(train_data, train_labels)

eval_metrics(train_labels, clf.predict(train_data))
eval_metrics(test_labels, clf.predict(test_data))

------------------------- EVALUATION -------------------------
Accuracy Score: 0.562430286727
             precision    recall  f1-score   support

          0       0.55      0.86      0.68      8041
          1       0.60      0.23      0.33      7200

avg / total       0.58      0.56      0.51     15241

Confusion Matrix:
[[6952 1089]
 [5580 1620]]
Kappa Score: 0.0925892224622
--------------------------------------------------------------
------------------------- EVALUATION -------------------------
Accuracy Score: 0.789200415369
             precision    recall  f1-score   support

          0       0.93      0.84      0.88       901
          1       0.04      0.10      0.06        62

avg / total       0.87      0.79      0.83       963

Confusion Matrix:
[[754 147]
 [ 56   6]]
Kappa Score: -0.0394314973441
--------------------------------------------------------------


In [22]:
""" Support Vector Machine """
from sklearn.svm import SVC
from sklearn import metrics

clf = SVC()
clf.fit(train_data, train_labels)

eval_metrics(train_labels, clf.predict(train_data))
eval_metrics(test_labels, clf.predict(test_data))

------------------------- EVALUATION -------------------------
Accuracy Score: 0.962272816744
             precision    recall  f1-score   support

          0       0.99      0.94      0.96      8041
          1       0.93      0.99      0.96      7200

avg / total       0.96      0.96      0.96     15241

Confusion Matrix:
[[7526  515]
 [  60 7140]]
Kappa Score: 0.924564478398
--------------------------------------------------------------
------------------------- EVALUATION -------------------------
Accuracy Score: 0.830737279335
             precision    recall  f1-score   support

          0       0.94      0.88      0.91       901
          1       0.07      0.13      0.09        62

avg / total       0.88      0.83      0.85       963

Confusion Matrix:
[[792 109]
 [ 54   8]]
Kappa Score: 0.00570092925147
--------------------------------------------------------------


In [18]:
""" OneClassSVM """
from sklearn.svm import OneClassSVM
from sklearn import metrics

clf = OneClassSVM(nu=0.1)
clf.fit(train_data)

eval_metrics(train_labels*-2+1, clf.predict(train_data))
eval_metrics(test_labels*-2+1, clf.predict(test_data))

------------------------- EVALUATION -------------------------
Accuracy Score: 0.855416030982
             precision    recall  f1-score   support

         -1       0.10      0.19      0.13       480
          1       0.95      0.90      0.92      8041

avg / total       0.90      0.86      0.88      8521

Confusion Matrix:
[[  92  388]
 [ 844 7197]]
Kappa Score: 0.059934937893
--------------------------------------------------------------
------------------------- EVALUATION -------------------------
Accuracy Score: 0.779854620976
             precision    recall  f1-score   support

         -1       0.08      0.23      0.12        62
          1       0.94      0.82      0.87       901

avg / total       0.88      0.78      0.83       963

Confusion Matrix:
[[ 14  48]
 [164 737]]
Kappa Score: 0.0234013240978
--------------------------------------------------------------


In [ ]:
x = np.random.normal(scale=0.1, size=(1000))
import scipy.stats as stats
n = stats.norm.pdf(x, np.mean(x), np.std(x))
print(np.mean(x), np.std(x))
plt.plot(x,n, '.')